In [1]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import uuid

import json
import urllib.parse as urlparse
from urllib.parse import parse_qs
from urllib.parse import urlparse
from urllib.request import urlopen
import time
import re

In [2]:
my_header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
              + "(KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36"}

querystring = {"page":"20", 'product_url':'https://www.walmart.com/search/?query=watch'}
item_url=[]
brand_name=[]
prod_name=[]
price=[]
free_delivery=[]
seller=[]
avg_rating=[]
additional_tags=[]
total_pages=int(querystring["page"])

avg_rating_per_page=[]

for i in range(1,total_pages+1):

    url='https://www.walmart.com/search/?page='+ str(i) +'?query=watch'
    r= requests.get(querystring['product_url'], headers= my_header)

    if r.status_code==200:
        soup_main= BeautifulSoup(r.text, parser='html.parser')
        #print(soup_main)
        summary=soup_main.find('div', {'class':'search-product-result', 'id':'searchProductResult'})
        product_list= summary.find_all('li')
        for prod in product_list:
            try:
                item_url.append(prod.find('a', {"class":"product-title-link line-clamp line-clamp-2 truncate-title"}).get('href'))
                brand_name.append(prod.find('a', {"class":"product-title-link line-clamp line-clamp-2 truncate-title"}).span.text)
                #getting product name
                prod_name.append(prod.find('a', {"class":"product-title-link line-clamp line-clamp-2 truncate-title"}).span.nextSibling.text)
                #getting price
                price.append(prod.find('span',{'class':'price-main-block'}).span.span.text.split('$')[1])
                #average rating
                avg_rating.append(float(prod.find('span',{'class':'visuallyhidden seo-avg-rating'}).text))
                
                try:
                    #getting seller name if any
                    seller.append(prod.find('span',{'class':"marketplace-sold-by-company-name"}).text)
                except AttributeError:
                    seller.append("NA")
                
                #checking if there is free delivery
                if(prod.find('div',{'class':'free-shipping'})):
                        free_delivery.append('True')

                else:
                    free_delivery.append('False')
                #adding additional tags such as "best seller", "Reduced price" etc
                try:
                    additional_tags.append(prod.find('span',{'class':'flag-angle__content'}).text)
                except:
                    additional_tags.append("NA")
            except:
                pass
        avg_rating_per_page.append(np.mean(np.array(avg_rating)))
                    
    else:
        print("Error-",r.status_code)


In [3]:
product_code=[u.split('/')[-1] for u in item_url]


In [4]:
print(len(item_url))
print(len(brand_name))
print(len(price))
print(len(free_delivery))
print(len(seller))
print(len(avg_rating))
print(len(additional_tags))
print(len(avg_rating_per_page))

200
200
200
200
200
200
200
20


In [5]:
item_dict={}
item_dict["Brand Name"]= brand_name
item_dict["Product Name"]= prod_name
item_dict["Product Price ($)"]= price
item_dict["Average Rating"]= avg_rating
item_dict["Availability of Free Delivery"]= free_delivery
item_dict["Seller Name"]= seller
item_dict["Additional Tags"]= additional_tags

df_products=pd.DataFrame(item_dict)
df_products.head(10)

,Brand Name,Product Name,Product Price ($),Average Rating,Availability of Free Delivery,Seller Name,Additional Tags
0,iTouch,iTouch Sport Silicone Strap Smartwatches with ...,59.99,4.4,True,NA,NA
1,Casio,Casio Women's Casual Classic Analog Watches LQ139,10.98,4.0,False,NA,Rollback
2,iTouch,iTouch Slim Fitness Trackers with Interchangea...,29.99,4.2,False,NA,NA
3,iTouch,iTouch Sport 2 Smartwatch Fitness Tracker Body...,59.99,4.6,True,NA,NA
4,iTouch,iTouch Air 3 Smartwatch Fitness Tracker Heart ...,59.99,4.7,True,NA,NA
5,Timex,"Men's Ironman Classic 30 Oversized Watch, Blac...",35.70,4.6,True,InfoSales,NA
6,Casio,Casio Men's Classic Resin Strap Sport Watch F9...,12.95,4.1,True,NSA ELECTRONICS,Best Seller
7,Geneva,Elecmall Stylish Stainless Steel Analog Quartz...,6.99,3.4,False,fulfilled by Walmart,Clearance
8,Zodaca,White Unisex Men Women Silicone Jelly Quartz A...,6.99,3.8,True,eForCity,NA
9,Casio,Classic Analog Water Resistant Watch w/ Resin ...,11.99,4.5,True,AreaTrend,Reduced Price


In [6]:
def preprocessor_final2(text):
        if isinstance((text), (str)):
            text = re.sub('<[^>]*>', ' ', text)
                #text = re.sub('[\W]+', ' ', text.lower())
            return text
        if isinstance((text), (list)):
            return_list = []
            for i in range(len(text)):
                temp_text = re.sub('<[^>]*>', '', text[i])
                temp_text = re.sub('[\W]+', '', temp_text.lower())
                return_list.append(temp_text)
            return(return_list)

In [7]:
reviews_dict={}
reviews_list=[]
reviews=[]

for prod_code in product_code:
    total_review_count=0

    item_url='https://www.walmart.com/reviews/product/'+ prod_code
    
    req= requests.get(item_url,headers=my_header)
    if req.status_code==200:
            prod_soup=BeautifulSoup(urlopen(item_url),parser="html.parser")
            review_list=prod_soup.findAll('div',{'class':'Grid ReviewList-content'})
            total_review_count+=len(review_list)
            averageOverallRating = prod_soup.find('div', {'class',"wyr-product-review-rounded-overall"}).get_text()
            
            for i in range(len(review_list)):
                    temp_dict = {}
                    temp_dict["totalReviewCount"] = total_review_count
                    temp_dict["product_id"]=prod_code
                    temp_dict["average_rating"]=averageOverallRating
                    try:
                        temp_dict["full_text"] = preprocessor_final2(str(review_list[i].find_all('div',{'class','review-text'})[0])).strip()
                    except AttributeError: 
                        temp_dict["full_text"] = preprocessor_final2(str(review_list[i].find_all('div',{'class','review-body'})[0])).strip()
                    except:
                        temp_dict["full_text"]='NA'
                    
                    
                    try:
                        temp_dict['title'] = preprocessor_final2(str(review_list[i].find_all('h3', {'class', 'review-title font-bold'})[0])).strip()
                    
                    except:
                        temp_dict['title'] = 'NA'
                    
                    try:
                        temp_dict['review_date'] = preprocessor_final2(str(review_list[i].find('div',{'class', 'review-date'}).find('span', {'class','review-date-submissionTime'}))).strip()                 
                    except:
                        temp_dict['review_date']='NA'
                    
                    try:
                        if review_list[i].find('div',{'class', 'review-date'}).find('div',{'class':'verified'}).get_text() == 'Verified purchase':
                            temp_dict["Verified"]="True"
                        
                        else:
                            temp_dict["Verified"]="False"
                    except:
                        pass
                    val=review_list[i].find_all('span', {'class','yes-no-count underline'})
                    temp_dict["Helpful-Yes"]=val[0].text
                    temp_dict["Helpful-No"]=val[1].text
                    reviews_list.append(temp_dict)
    else:
            print("Error-",req.status_code)
    reviews_dict["reviews"] = reviews_list 

In [8]:
df = pd.DataFrame(reviews_dict["reviews"])
df=df.set_index(['product_id'])
df.head()

,totalReviewCount,average_rating,full_text,title,review_date,Verified,Helpful-Yes,Helpful-No
product_id,,,,,,,,
173019570,20,4.4,It's really nice and easy to understand. It te...,The best smart watch I've had!,"December 9, 2020",True,0,0
173019570,20,4.4,I shopped around for a smart watch online not ...,Love my watches!,"August 24, 2020",True,0,0
173019570,20,4.4,Pretty good watch. Not and iwatch of course so...,Very nice watch,"September 30, 2020",True,0,0
173019570,20,4.4,"If you want to keep up with step , but are not...",Keeping Up!,"October 15, 2020",True,0,0
173019570,20,4.4,"I am loving this watch! It is so clean, simple...",Chic &amp; Cool,"April 9, 2020",True,0,0


In [9]:
'''df_products.to_csv("Watches_info.csv")
df.to_csv("Watches_reviews.csv")'''

'df_products.to_csv("Watches_info.csv")\ndf.to_csv("Watches_reviews.csv")'

## Explatory Data Analysis

### 1. Average Rating on each page

In [10]:
avg_rating_per_page

[4.2299999999999995,
 4.23,
 4.23,
 4.2299999999999995,
 4.23,
 4.23,
 4.23,
 4.2299999999999995,
 4.2299999999999995,
 4.23,
 4.23,
 4.23,
 4.23,
 4.23,
 4.23,
 4.2299999999999995,
 4.2299999999999995,
 4.23,
 4.23,
 4.23]

 we can see that the average rating on each page is the same

### 2. Average word count of reviews

In [11]:
def word_count(txt):
    count=len(txt.split(' '))
    return count

df["review word count"]=df["full_text"].apply(word_count)
average_word_count= int(np.mean(df["review word count"]))
average_word_count

63

The average word count of the reviews are 62 words

### 3. Ratio of verified purchase to total reviews

In [12]:
df["Verified"]=df["Verified"].fillna("False")

In [13]:
verified_count=df["Verified"][df["Verified"]=="True"].count()

In [14]:
#ratio
verified_count/len(df["Verified"])

0.66

the ratio of verified to total reviews is 0.62

### 4. Average age of reviews

In [15]:
import datetime
df["review_date"]=pd.to_datetime(df["review_date"])
date_today=pd.to_datetime("December 11, 2020")

In [16]:
df["age of review"]= date_today-df["review_date"]
df.head()

,totalReviewCount,average_rating,full_text,title,review_date,Verified,Helpful-Yes,Helpful-No,review word count,age of review
product_id,,,,,,,,,,
173019570,20,4.4,It's really nice and easy to understand. It te...,The best smart watch I've had!,2020-12-09,True,0,0,50,2 days
173019570,20,4.4,I shopped around for a smart watch online not ...,Love my watches!,2020-08-24,True,0,0,219,109 days
173019570,20,4.4,Pretty good watch. Not and iwatch of course so...,Very nice watch,2020-09-30,True,0,0,44,72 days
173019570,20,4.4,"If you want to keep up with step , but are not...",Keeping Up!,2020-10-15,True,0,0,42,57 days
173019570,20,4.4,"I am loving this watch! It is so clean, simple...",Chic &amp; Cool,2020-04-09,True,0,0,77,246 days


In [17]:
np.mean(df["age of review"].dt.days)

484.76

average age of a review is around 484 days

### 5. Count of reviews being helpful vs not helpful

In [18]:
df["Helpful-Yes"]=pd.to_numeric(df["Helpful-Yes"])
sum(df["Helpful-Yes"])

6020

In [19]:
df["Helpful-No"]=pd.to_numeric(df["Helpful-No"])
sum(df["Helpful-No"])

980